In [152]:
from xirr import xirr 
import datetime as datetime
from prefutils import *

import pandas_datareader as pdr
import datetime 
import pandas as pd
import numpy as np

import pandas_datareader.data as web
import requests_cache
import matplotlib.pyplot as plt
from datetime import date

In [149]:
gwo_price = 13.58
gwo_spread = 130
gwo_reset_date = datetime.datetime(2020,12,31)
gwo_dividend_annual = 0.544

estimated_goc5_at_reset_decimal = 0.02

resets_maturity_date = datetime.datetime(2040,12,15)
gwo_months = [3, 6, 9, 12]

def dividend_after_reset(issue_reset_spread_bips, goc5_at_reset_decimal, par=25) :
    yield_at_reset = (estimated_goc5_at_reset_decimal + issue_reset_spread_bips/10000) 
    return par*yield_at_reset


gwo_reset_dividend_annual = dividend_after_reset(gwo_spread, estimated_goc5_at_reset_decimal)




In [146]:

# for now, ignore market holidays
def payment_day_last_of_month(year, month) :
    days_in_month = monthrange(year,month)[1]
    return datetime.datetime(year,month,days_in_month)

def next_month(x) :
    return 1 if x>= 12 else x+1

def next_payment_date_after_date(after_date, month_list) :
    month = after_date.month
    # if on last day of month, then search next month
    if after_date.day == monthrange(after_date.year,month)[1] :
        month = next_month(month)
    
    # search the next month
    while (not month in month_list) :
        month = next_month(month)
    
    # found month but it could be following year
    which_year = after_date.year if (month >= after_date.month) else after_date.year+1
    
    return payment_day_last_of_month(which_year, month)


def build_cashflow_list(purchase_date, purchase_price, reset_date, maturity_date, payment_months, 
                        current_dividend_annual, reset_dividend_annual) :
    group = []
    next_date = next_payment_date_after_date(purchase_date, payment_months)

    while (next_date < maturity_date) :
        group.append(next_date)
        next_date = next_payment_date_after_date(next_date, payment_months)   
    
    # walk date list; if before reset, use current, otherwise reset
    div_q = current_dividend_annual/4
    pr_div_q = reset_dividend_annual/4

    def which_payment(pbefore, pafter, curdate, breakdate) :
        return pbefore if curdate <= breakdate else pafter

    cashflows = [ (x, which_payment(div_q, pr_div_q, x, reset_date)) for x in group ]

    def add_initial_purchase_cashflow(date_cash_list, purchase_date, purchase_price):
        date_cash_list.insert(0,(purchase_date, -1 * purchase_price))
    
    def add_maturity_sale_cashflow(date_cash_list, maturity_date, sale_price):
        date_cash_list.append((maturity_date, sale_price))

    add_initial_purchase_cashflow(cashflows, purchase_date, purchase_price)
    add_maturity_sale_cashflow(cashflows, resets_maturity_date, purchase_price)

    return cashflows



In [150]:

cur_date = datetime.datetime.today() 
flows = build_cashflow_list(cur_date, gwo_price, gwo_reset_date, resets_maturity_date, gwo_months,
                           gwo_dividend_annual, gwo_reset_dividend_annual)

xirr(flows, 0.2)

0.06015945410476384

In [156]:
# Next Steps
#
# 1. read in CSV table of fixed resets; update prices
# 2. compute the current market spread
# 3. compute future dividend (add to table)
# 4. given a market spread, compute price as a function of various GOC5 estimate
# 5. loop over multiple MS scenarios. Unlike specifying explicitly, use functional guess

SESSION = init_fetch_session()
resets = pd.read_csv('./resets2.csv', dtype={'Notes': object})
resets

,Ticker,Rating,Div,Spread,Reset Date,Price,Notes
0,AIM.PR.A,ZR,$1.12500,375,03/2020,$16.55,NaN
1,AIM.PR.C,ZR,$1.50250,420,03/2024,$19.48,NaN
2,BAM.PR.G,P2L,$0.68750,230,11/2021,$14.52,NaN
3,EMA.PR.A,P3H,$0.63875,184,08/2020,$14.15,NaN
4,PWF.PR.P,P2H,$0.57650,160,01/2021,$13.85,NaN
...,...,...,...,...,...,...,...
169,RY.PR.H,P2In,$0.91250,226,08/2024,$17.66,NVCC
170,BMO.PR.T,P2In,$0.90600,224,08/2024,$17.48,NVCC
171,FTS.PR.M,P3H,$1.02500,248,12/2024,$17.51,NaN
172,TRP.PR.B,P2L,$0.53800,128,06/2020,$12.08,NaN


In [157]:
df = resets.drop(columns=['Notes'],inplace=True,errors='ignore')

In [160]:
df = resets.copy()


In [161]:
df

,Ticker,Rating,Div,Spread,Reset Date,Price
0,AIM.PR.A,ZR,$1.12500,375,03/2020,$16.55
1,AIM.PR.C,ZR,$1.50250,420,03/2024,$19.48
2,BAM.PR.G,P2L,$0.68750,230,11/2021,$14.52
3,EMA.PR.A,P3H,$0.63875,184,08/2020,$14.15
4,PWF.PR.P,P2H,$0.57650,160,01/2021,$13.85
...,...,...,...,...,...,...
169,RY.PR.H,P2In,$0.91250,226,08/2024,$17.66
170,BMO.PR.T,P2In,$0.90600,224,08/2024,$17.48
171,FTS.PR.M,P3H,$1.02500,248,12/2024,$17.51
172,TRP.PR.B,P2L,$0.53800,128,06/2020,$12.08
